In [2]:
label='returnQuantity'

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from scipy import stats
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder

def dmc_error(returnQuantity, prediction):
    #∑ |returnQuantity𝑖 − prediction𝑖 |
    return np.sum(np.abs( np.subtract(returnQuantity, prediction) ))

In [3]:
df=pd.read_csv('train_gue3.csv')

In [4]:
df = df[:1023297]

In [5]:
object_columns=df.loc[:, df.dtypes == object].columns
df= df.drop(object_columns, axis=1)

In [6]:
df= df.drop(['colorCode','productGroup','months','price','rrp','voucherAmount','after_voucher'], axis=1)

In [7]:
label='returnQuantity'

Xdf=df.drop([label], axis=1)

X=Xdf.values
y=df.returnQuantity.values

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [8]:
lr=LinearRegression(n_jobs=-1, normalize=True)
lr.fit(X_train, y_train)
y_pred=lr.predict(X_test)
y_pred=np.array([round(yp) for yp in y_pred], dtype=np.int8)
y_pred[y_pred<0]=0
print confusion_matrix(y_test,y_pred)
print dmc_error(y_test,y_pred)

[[147407  15305      7      1      1      0]
 [ 13456 160924      5      0      0      0]
 [     1    538      5      0      0      0]
 [     0      7     23      0      0      0]
 [     0      0      4      0      0      0]
 [     0      0      5      0      0      0]]
29387


In [9]:
lr=linear_model.Lasso()
lr.fit(X_train, y_train)
y_pred=lr.predict(X_test)
y_pred=np.array([round(yp) for yp in y_pred], dtype=np.int8)
y_pred[y_pred<0]=0
print confusion_matrix(y_test,y_pred)
print dmc_error(y_test,y_pred)

[[106510  56206      5      0      0      0]
 [ 74670  99715      0      0      0      0]
 [   189    355      0      0      0      0]
 [    11     19      0      0      0      0]
 [     0      4      0      0      0      0]
 [     1      4      0      0      0      0]]
131723


In [10]:
lr=linear_model.Ridge()
lr.fit(X_train, y_train)
y_pred=lr.predict(X_test)
y_pred=np.array([round(yp) for yp in y_pred], dtype=np.int8)
y_pred[y_pred<0]=0
print confusion_matrix(y_test,y_pred)
print dmc_error(y_test,y_pred)

[[147400  15312      7      1      1      0]
 [ 13450 160930      5      0      0      0]
 [     1    538      5      0      0      0]
 [     0      7     23      0      0      0]
 [     0      0      4      0      0      0]
 [     0      0      5      0      0      0]]
29388


In [11]:
poly = PolynomialFeatures(2)
X = poly.fit_transform(X).astype(np.float32)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [13]:
lr=LinearRegression(n_jobs=-1, normalize=True)
lr.fit(X_train, y_train)
y_pred=lr.predict(X_test)
y_pred=np.array([round(yp) for yp in y_pred], dtype=np.int8)
y_pred[y_pred<0]=0
print confusion_matrix(y_test,y_pred)
print dmc_error(y_test,y_pred)

[[146612  16101      7      1      0      0]
 [ 11733 162577     74      0      1      0]
 [     6    104    434      0      0      0]
 [     0      2      6     22      0      0]
 [     0      0      0      1      3      0]
 [     0      0      1      1      1      2]]
28061


In [14]:
lr=linear_model.Lasso()
lr.fit(X_train, y_train)
y_pred=lr.predict(X_test)
y_pred=np.array([round(yp) for yp in y_pred], dtype=np.int8)
y_pred[y_pred<0]=0
print confusion_matrix(y_test,y_pred)
print dmc_error(y_test,y_pred)

[[149394  13327      0      0      0      0]
 [ 21497 152885      3      0      0      0]
 [    66    474      4      0      0      0]
 [     2     26      2      0      0      0]
 [     0      4      0      0      0      0]
 [     0      4      1      0      0      0]]
35524


In [15]:
lr=linear_model.Ridge()
lr.fit(X_train, y_train)
y_pred=lr.predict(X_test)
y_pred=np.array([round(yp) for yp in y_pred], dtype=np.int8)
y_pred[y_pred<0]=0
print confusion_matrix(y_test,y_pred)
print dmc_error(y_test,y_pred)

[[146615  16098      7      1      0      0]
 [ 11756 162555     73      0      1      0]
 [     6    104    434      0      0      0]
 [     0      2      6     22      0      0]
 [     0      0      0      1      3      0]
 [     0      0      1      1      1      2]]
28080


In [16]:
df.columns

Index([u'quantity', u'returnQuantity', u'order_total', u'customer_budget',
       u'expense_ratio', u'unit_price', u'usual_unit_price', u'price_diff',
       u'order_order', u'choice_order', u'articleID_cumprob',
       u'colorCode_cumprob', u'customerID_cumprob', u'sizeCode_cumprob',
       u'ac_cumprob', u'as_cumprob', u'cps_cumprob'],
      dtype='object')

In [8]:
def time_series_validation(n, step):
    """Cross validation for time-series data."""
    leap = 10000; k = 150000; h = 30000
    indices = np.arange(n)
    for i in range(step):
        yield indices[:k], indices[k:k+h]
        k += leap

In [9]:
def validation(X,y):
    """Local Cross Validation.
    """
    X_train= X
    y_train= y
    regress = LinearRegression(n_jobs=-1, normalize=True)

    print(" --- Start local evaluation.")
    tsv = time_series_validation(X_train.shape[0], 33)
    i=1; scores = []
    for train, test in tsv:
        size = test.shape[0]

        regress.fit(X_train[train], y_train[train])
        y_pred = regress.predict(X_train[test])
        y_pred = np.array([round(yp) for yp in y_pred], dtype=np.int8)
        y_pred[y_pred < 0] = 0

        y_true = y_train[test]
        
        score = dmc_error(y_true, y_pred)

        print(" ---- Score of #{0} : {1:.5f}".format(i, score))
        i += 1
        scores.append(score)

    print(" --- Finished local evaluation.")
    quantiles = stats.mstats.mquantiles(scores)
    print(" --- Score Results:")
    print(" - min: {:.5f}".format(np.min(scores)))
    print(" - 25%: {:.5f}".format(quantiles[0]))
    print(" - median: {:.5f}".format(np.median(scores)))
    print(" - 75%: {:.5f}".format(quantiles[2]))
    print(" - max: {:.5f}".format(np.max(scores)))
    print(" - mean: {0:.5f} (+/-{1:.5f})".format(np.mean(scores), stats.sem(scores)))

In [10]:
validation(X,y)

 --- Start local evaluation.
 ---- Score of #1 : 2123
 ---- Score of #2 : 2123
 ---- Score of #3 : 2202
 ---- Score of #4 : 2196
 ---- Score of #5 : 2162
 ---- Score of #6 : 2126
 ---- Score of #7 : 2154
 ---- Score of #8 : 2214
 ---- Score of #9 : 2371
 ---- Score of #10 : 2426
 ---- Score of #11 : 2550
 ---- Score of #12 : 2501
 ---- Score of #13 : 2548
 ---- Score of #14 : 2543
 ---- Score of #15 : 2518
 ---- Score of #16 : 2571
 ---- Score of #17 : 2515
 ---- Score of #18 : 2584
 ---- Score of #19 : 2531
 ---- Score of #20 : 2591
 ---- Score of #21 : 2562
 ---- Score of #22 : 2545
 ---- Score of #23 : 2523
 ---- Score of #24 : 2515
 ---- Score of #25 : 2550
 ---- Score of #26 : 2661
 ---- Score of #27 : 2766
 ---- Score of #28 : 2903
 ---- Score of #29 : 2908
 ---- Score of #30 : 2981
 ---- Score of #31 : 2863
 ---- Score of #32 : 2907
 ---- Score of #33 : 2785
 --- Finished local evaluation.
 --- Score Results:
 - min: 2123
 - 25%: 2323.9
 - median: 2543
 - 75%: 2612
 - max: 2981
